<a href="https://colab.research.google.com/github/rajeshr6r/Python_Assignments/blob/master/COVID_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import requests
import csv
import datetime
import pandas as pd
from io import StringIO

"""
Designed to download Vaccination , Infection , Death and Recovery rate for COVID 

Vaccination Data comes from : https://github.com/owid/covid-19-data/tree/master/public/data/vaccinations 
TimeSerios for Infection,Death and Recovery comes from  : :https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

These data sets have US and Global data . So possible duplications .

These data sets are downloaded somewhere around 5:30 AM EST ( safer side to download them at about 0930 EST)

We create a csv file for each url with a timestamp to denote the time at which the script got executed at our end 

Total files after script execution should be : 8

"""

In [ ]:
#url initialization
vaccinations_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
vaccinations_by_age_group_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-age-group.csv"
vaccinations_by_manuf_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-manufacturer.csv"
covid19_timeseries_confirmed_us_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
covid19_timeseries_confirmed_global_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
covid19_time_series_covid19_deaths_US_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
covid19_time_series_covid19_deaths_global_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
covid19_time_series_covid19_recovered_global_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [ ]:
#function to get data 
def process_url(url_to_be_processed,file_name_for_output):
    try:
        req = requests.get(url_to_be_processed)
        url_content = req.content
        #csv_file = open(file_name_for_output, "wb")
        #csv_file.write(url_content)    
        #csv_file.close()

        result = str(url_content, 'utf-8')
        data = StringIO(result)
        dataframe_variable = pd.read_csv(data)
        return dataframe_variable
    except Exception as e:
        print(f"Exception occured {str(e)}")
        return pd.DataFrame()


In [ ]:
#download and save the files and also load them into a dataframe

df_vacc=process_url(vaccinations_url,'vaccinations.csv')
df_vac_by_age=process_url(vaccinations_by_age_group_url,'vaccinations_by_age_group.csv')
df_vacc_by_manuf=process_url(vaccinations_by_manuf_url,'vaccinations_by_manufacturer.csv')
df_timeseries_confirmed_us=process_url(covid19_timeseries_confirmed_us_url,'covid19_timeseries_confirmed_US.csv')
df_timeseries_confirmed_global=process_url(covid19_timeseries_confirmed_global_url,'covid19_timeseries_confirmed_global.csv')
df_timeseries_death_US=process_url(covid19_time_series_covid19_deaths_US_url,'covid19_time_series_covid19_deaths_US.csv')
df_timeseries_death_global=process_url(covid19_time_series_covid19_deaths_global_url,'covid19_time_series_covid19_deaths_global.csv')
df_timeseries_recovered_global=process_url(covid19_time_series_covid19_recovered_global_url,'covid19_time_series_covid19_recovered_global.csv')

In [ ]:
#transpose the timeseries data to get the date wise data in rows from columns

df_timeseries_confirmed_global_t=df_timeseries_confirmed_global.melt(['Province/State', 'Country/Region', 'Lat', 'Long'], var_name='ReportDate')
df_timeseries_death_global_t=df_timeseries_death_global.melt(['Province/State', 'Country/Region', 'Lat', 'Long'], var_name='ReportDate')
df_timeseries_recovered_global_t=df_timeseries_recovered_global.melt(['Province/State', 'Country/Region', 'Lat', 'Long'], var_name='ReportDate')


In [ ]:
#Cleanup of column names post transpose

df_timeseries_confirmed_global_t=df_timeseries_confirmed_global_t.rename(columns = {'value': 'Confirmed'}, inplace = False)
df_timeseries_death_global_t=df_timeseries_death_global_t.rename(columns = {'value': 'Deaths'}, inplace = False)
df_timeseries_recovered_global_t=df_timeseries_recovered_global_t.rename(columns = {'value': 'Recovered'}, inplace = False)

df_timeseries_confirmed_global_t['ReportDate']=pd.to_datetime(df_timeseries_confirmed_global_t['ReportDate'])
df_timeseries_death_global_t['ReportDate']=pd.to_datetime(df_timeseries_death_global_t['ReportDate'])
df_timeseries_recovered_global_t['ReportDate']=pd.to_datetime(df_timeseries_recovered_global_t['ReportDate'])


In [ ]:
#create a merged data frame to map confirmed / recovered / death cases side by side

final_df = pd.merge(df_timeseries_confirmed_global_t, df_timeseries_death_global_t,  how='left', left_on=['Province/State', 'Country/Region', 'Lat', 'Long','ReportDate'], right_on = ['Province/State', 'Country/Region', 'Lat', 'Long','ReportDate'])
final_df = pd.merge(final_df, df_timeseries_recovered_global_t,  how='left', left_on=['Province/State', 'Country/Region', 'Lat', 'Long','ReportDate'], right_on = ['Province/State', 'Country/Region', 'Lat', 'Long','ReportDate'])


In [ ]:
#sort for obtaining difference between successive days to get day wise numbers .
#Catch is that this works well with the data being sorted against the country and date . 
#However it blindly gets the difference without taking into consideration the difference in the country name.
#So the first record for each country is not accurate . we should simply ignore the first record as a ugly fix.

final_timeseries_df=final_df.sort_values(['Country/Region', 'ReportDate'], ascending=[True, True])

In [ ]:
final_timeseries_output_df=pd.concat([final_timeseries_df,final_timeseries_df[['Confirmed', 'Deaths' , 'Recovered']].diff().rename({'Confirmed':'ConfirmedDiff', 'Deaths':'DeathDiff','Recovered':'RecoveredDiff'}, axis=1)], axis=1)

In [ ]:
#drop data for 22-01-2020 as this is not required . Refer to the previous step point number 4 for reasons. 
#Post a validation of the data on 07-Jul-2021 and finding that after deletion of this date the cumulative numbers are matching with that the derived daily numbers ( based on the difference between 2 successive days )
final_timeseries_output_df.drop(final_timeseries_output_df[final_timeseries_output_df['ReportDate'] == '2020-01-22T00:00:00.000000000'].index, inplace = True)

In [ ]:
final_timeseries_output_df.to_csv('final_timeseries_data.csv', index=False)

In [ ]:
#Vaccination Data Treatment
#adding a dummy entry with 0 as starting counter to deal with countries that start with a number on day 1 
for country in list(pd.unique(df_vacc['location'])):
    df_vacc=df_vacc.append(
        {
            'location':country,
            'iso_code':'',
            'date':'2020-01-01',
            'total_vaccinations':0,
            'people_vaccinated':0,
            'people_fully_vaccinated':0,
            'daily_vaccinations_raw':0,
            'daily_vaccinations':0,
            'total_vaccinations_per_hundred':0,
            'people_vaccinated_per_hundred':0,
            'people_fully_vaccinated_per_hundred':0,
            'daily_vaccinations_per_million':0
        },
        ignore_index=True
    )



In [ ]:
df_vacc=df_vacc.reset_index(drop=True)
df_vacc['date']=pd.to_datetime(df_vacc['date'])
df_vacc=df_vacc.sort_values(['location', 'date'], ascending=[True, True])

In [ ]:
#function to treat country wise data and generate a clean data frame for day level calculations while populating missing values based on previously available data whenever possible for the same country
def datacleanup(inputdataframe,countryname):
    idf=inputdataframe
    #first filter the dataframe to only have the relevant country . because the ffill conditions doesnt work great
    idf=idf[idf['location']==countryname]
    idf['total_vaccinations_new']=idf['total_vaccinations']
    idf['people_vaccinated_new']=idf['people_vaccinated']
    idf['people_fully_vaccinated_new']=idf['people_fully_vaccinated']
    idf['total_vaccinations_new'].fillna(method='ffill',axis=0,inplace=True)
    idf['people_vaccinated_new'].fillna(method='ffill',axis=0,inplace=True)
    idf['people_fully_vaccinated_new'].fillna(method='ffill',axis=0,inplace=True)
    #an additional step to avoid 0 and na problems
    idf['total_vaccinations_new'].fillna(0,inplace=True)
    idf['people_vaccinated_new'].fillna(0,inplace=True)
    idf['people_fully_vaccinated_new'].fillna(0,inplace=True)
    
    #logic for calculating difference between successive days 
    odf=pd.concat([idf,idf[['total_vaccinations_new', 'people_vaccinated_new' , 'people_fully_vaccinated_new']].diff().rename({'total_vaccinations_new':'total_vaccinations_diff', 'people_vaccinated_new':'people_vaccinated_diff','people_fully_vaccinated_new':'people_fully_vaccinated_diff'}, axis=1)], axis=1)
    
    
    
    return odf

In [ ]:
#As discussed with Puja and Ashu on July 8th 2021
countries_for_vaccination_processing=list(pd.unique(df_vacc['location']))

In [ ]:
finaldf=pd.DataFrame()
for country in countries_for_vaccination_processing:
    tempdf=datacleanup(df_vacc,country)
    finaldf=finaldf.append(tempdf)
    
print("Done")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

Done


In [ ]:
finaldf.to_csv('final_vaccinations_data.csv', index=False)